In [7]:
import requests, json
from bs4 import BeautifulSoup as BS
import pandas as pd
import geocoder
import numpy as np
import re
from country_codes import country_codes, alt_country

In [8]:
def country_name_lookup(df, dictionary):
    for country in dictionary:   # for each key in a dictionary whos keys are country names
        for abbr in df['Country']: # and for each element in the Country column of the df provided
            if abbr in dictionary[country]: # if the element is in the list of abbreviations for that country provided by the dictionary
                df.loc[df['Country']==abbr, 'Country'] = country  # observations in the country column with a value  == the element are set to the country name
    return(df)

In [9]:

locations = 'https://developers.google.com/public-data/docs/canonical/countries_csv'



loc_data = requests.get(locations).content

loc_data = re.sub(r'<.*?>', lambda g: g.group(0).upper(), str(loc_data))
BS(loc_data, 'html.parser')

loc_html = BS(loc_data, 'html.parser')
loc_table = loc_html.find_all('table')[0]

pd.read_html(str(loc_table))

meta_loc_df = pd.read_html(str(loc_table))

loc_records = meta_loc_df[0].to_json(orient='records')
loc_data = json.loads(loc_records)
covid_loc_df= pd.read_json(json.dumps(loc_data))


In [10]:
rec_url = 'https://www.worldometers.info/coronavirus/#countries'

rec_data = requests.get(rec_url).content

rec_data = re.sub(r'<.*?>', lambda g: g.group(0).upper(), str(rec_data))
BS(rec_data, 'html.parser')

rec_html = BS(rec_data, 'html.parser')
rec_table = rec_html.find_all('table')[0]

pd.read_html(str(rec_table))

meta_rec_df = pd.read_html(str(rec_table))

rec_records = meta_rec_df[0].to_json(orient='records')
rec_data = json.loads(rec_records)
covid_rec_df= pd.read_json(json.dumps(rec_data))

covid_rec_df =covid_rec_df.drop([0,1,2,3,4,5,6,7],)

recover_df = covid_rec_df.sort_values(by='Total Cases', ascending = False)
recover_df
recover_df.reset_index()
recover_df.dropna(subset=['#'],inplace= True)


recover_df.rename(columns={'Country, Other':'Country',
                          'Total Recovered': 'Recoveries',
                          'Total Cases':'Cases',
                          'Total Deaths':'Deaths',
                           '#':'Rank',}, inplace = True)

country_name_lookup(recover_df, alt_country)
recover_df = recover_df.reset_index(drop=True)




In [11]:
country_code_df = pd.DataFrame(country_codes)
country_code_df.rename(columns={'name':'Country'}, inplace=True)

covid_loc_df.rename(columns={'name':'Country'}, inplace=True)


country_locs_df = covid_loc_df
full_df=pd.merge(recover_df, country_locs_df, on='Country')

full_df['Rank']=full_df['Rank'].astype(int)
code_df = pd.merge(full_df, country_code_df, on='Country')
flag_urls = []
for code in code_df['code']:
    flag_url = f'https://cdn.jsdelivr.net/npm/country-flag-icons@1.5.7/1x1/{code}.svg'
    flag_urls.append(flag_url)

flag_urls = pd.DataFrame(flag_urls)
flag_urls.columns = ['flag_url']
flag_urls['Country'] =code_df['Country']
full_df = pd.merge(flag_urls, full_df, on ='Country')
full_df = full_df.replace(np.nan, 0)

In [12]:
full_df

,flag_url,Country,Rank,Cases,New Cases,Deaths,New Deaths,Recoveries,New Recovered,Active Cases,...,Continent,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop,country,latitude,longitude
0,https://cdn.jsdelivr.net/npm/country-flag-icon...,United States,1,111820082,0.0,1219487.0,0.0,109814428.0,0.0,786167.0,...,North America,3,275.0,0.0,0.0,0.0,2348.0,US,37.090240,-95.712891
1,https://cdn.jsdelivr.net/npm/country-flag-icon...,India,2,45035393,0.0,533570.0,0.0,0.0,0.0,0.0,...,Asia,31,2636.0,2.0,0.0,0.0,0.4,IN,20.593684,78.962880
2,https://cdn.jsdelivr.net/npm/country-flag-icon...,France,3,40138560,0.0,167642.0,0.0,39970918.0,0.0,0.0,...,Europe,2,391.0,0.0,0.0,0.0,0.0,FR,46.227638,2.213749
3,https://cdn.jsdelivr.net/npm/country-flag-icon...,Germany,4,38828995,0.0,183027.0,0.0,38240600.0,0.0,405368.0,...,Europe,2,458.0,1.0,0.0,0.0,4833.0,DE,51.165691,10.451526
4,https://cdn.jsdelivr.net/npm/country-flag-icon...,Brazil,5,38743918,0.0,711380.0,0.0,36249161.0,0.0,1783377.0,...,South America,6,303.0,3.0,0.0,0.0,8281.0,BR,-14.235004,-51.925280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,https://cdn.jsdelivr.net/npm/country-flag-icon...,Saint Helena,222,2166,0.0,0.0,0.0,2.0,0.0,2164.0,...,Africa,3,0.0,0.0,0.0,0.0,353884.0,SH,-24.143474,-10.030696
191,https://cdn.jsdelivr.net/npm/country-flag-icon...,Montserrat,224,1403,0.0,8.0,0.0,1376.0,0.0,19.0,...,North America,4,621.0,0.0,0.0,0.0,3827.0,MS,16.742498,-62.187366
192,https://cdn.jsdelivr.net/npm/country-flag-icon...,Niue,225,1059,0.0,0.0,0.0,1056.0,0.0,3.0,...,Australia/Oceania,2,0.0,0.0,0.0,0.0,1850.0,NU,-19.054445,-169.867233
193,https://cdn.jsdelivr.net/npm/country-flag-icon...,Tokelau,227,80,0.0,0.0,0.0,0.0,0.0,80.0,...,Australia/Oceania,17,0.0,0.0,0.0,0.0,58055.0,TK,-8.967363,-171.855881
